In [1]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
from torch import cuda
from sql_metadata import Parser
from tqdm import tqdm

/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "/home/LLM_para/deepspeek_code"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2", # use with amper architecture
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config, # use when low on memory
    device_map = "auto"
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.28s/it]


In [3]:
adapter_path = "./final_checkpoint_SFT_deepspeek_Myidea_verify"
model = PeftModel.from_pretrained(model,adapter_path ,torch_dtype = torch.bfloat16)
model = model.merge_and_unload()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer.encode(' ;')

[32013, 6203]

In [6]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
#获取表数据
rec_tablename_pattern = r"CREATE TABLE `([^`]+)`"
# table_name = re.search(rec_tablename_pattern, item).group(1).lower()

df = pd.read_csv("mydataset_new/table_schema_Reference_cropped.csv",encoding='utf-8')

table_name_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    reference_tables = row['Reference_group']
    table_name =  re.search(rec_tablename_pattern, reference_tables).group(1)
    if table_name is None:
        print("error")
    table_name_list.append(table_name)


df = pd.read_csv("mydataset_new/table_schema_noReference_cropped.csv",encoding='utf-8')
for index, row in tqdm(df.iterrows(), total=len(df)):
    reference_tables = row['noReference_group']
    table_name = re.search(rec_tablename_pattern, reference_tables).group(1)
    if table_name is None:
        print("error")
    table_name_list.append(table_name)
# print(table_name_list)
# print(len(table_name_list))
table_name_str = ", ".join(table_name_list)
schema_str = "[" + table_name_str + "]"
print(table_name_str)
check_token = tokenizer(f"{schema_str}", add_special_tokens=False)    
print(len(check_token["input_ids"]))
max_token = 900

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 13772.89it/s]

Manufacturers, Products, Student, Plays_Games, SportsInfo, actor, entrepreneur, people, Ref_Hotel_Star_Ratings, Ref_Attraction_Types, Hotels, Tourist_Attractions, Street_Markets, Shops, Museums, Royal_Family, Theme_Parks, Visits, Photos, Staff, Tourist_Attraction_Features, wrestler, Elimination, business, category, checkin, neighbourhood, review, tip, Ref_Detention_Type, Ref_Incident_Type, Addresses, Students, Teachers, Assessment_Notes, Behavior_Incident, Detention, Student_Addresses, Students_in_Detention, film, film_market_estimation, Catalogs, Catalog_Structure, Catalog_Contents, Catalog_Contents_Additional_Attributes, routes, airports, stadium, game, injury_accident, Physician, Department, Affiliated_With, Trained_In, Patient, Nurse, Appointment, Prescribes, Block, Room, On_Call, Stay, Undergoes, buildings, Office_locations, region, party, member, party_events, Web_client_accelerator, accelerator_compatible_browser, Investors, Lots, Transactions, Sales, Purchases, Transactions_Lot

In [7]:
from transformers import StoppingCriteria
# from typing import List
# from transformers.generation.stopping_criteria import StoppingCriteria, StoppingCriteriaList,STOPPING_CRITERIA_INPUTS_DOCSTRING, add_start_docstrings
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [6203]):
    # 设置在第一个';'处停止      
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids
    
def append_string_to_file(text, file_path):
  with open(file_path, 'a') as file:
      file.write(text + '\n')

def remove_spaces(text):
  return re.sub(r'\s+', ' ', text)

def call_mistral(inputs):
    output_tokens = model.generate(inputs, max_new_tokens=40, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = [EosListStoppingCriteria()])
#     output_tokens = model.generate(inputs, max_new_tokens=250, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = stopping_criteria)
    return tokenizer.decode(output_tokens[0][len(inputs[0]):], skip_special_tokens=True)

In [8]:
def get_input_value(question_pad,schema_pad):
    input_value = f"I want you to act as a relation extraction robot for a sample SQL table. You need to return the tables related to the user's input question and tables.##Question:{question_pad}.Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}"
    return input_value 

In [10]:
df = pd.read_csv("mydataset_new/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
import time
start_time = time.time()
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
#  schema_slicing

for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    input_value = get_input_value(question,schema_str)
    messages = [{"role": "user", "content": input_value.strip()}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",add_generation_prompt=True,tokenize = True).to(model.device)
    response = call_mistral(inputs)
    match = re.search(pattern, response)
    if match:
    # 提取匹配的表名字符串
        table_names_str = match.group(1)
        table_names = table_names_str.split(', ')
        predict_table = ', '.join(table_names)
        print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
        print("预测表名：", predict_table)
        print("正确表名：", ref_tables)
        print("############################################")
    else:
        predict_table = "None"
    results.append([ref_tables,predict_table])
end_time = time.time()

  1%|▍                                                                                  | 1/178 [00:00<01:39,  1.77it/s]

++++++++++++++++index:0++++++++++++++++++++++++
预测表名： products, manufacturers
正确表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:01<01:30,  1.94it/s]

++++++++++++++++index:1++++++++++++++++++++++++
预测表名： products
正确表名： products
############################################


  2%|█▍                                                                                 | 3/178 [00:01<01:29,  1.95it/s]

...

++++++++++++++++index:175++++++++++++++++++++++++
预测表名： railway
正确表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [01:34<00:00,  2.17it/s]

++++++++++++++++index:176++++++++++++++++++++++++
预测表名： book
正确表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [01:34<00:00,  1.88it/s]

++++++++++++++++index:177++++++++++++++++++++++++
预测表名： book
正确表名： book, publication
############################################


In [11]:
total_data = len(df['question'])
total_time = end_time - start_time
# print(total_data)
signal_infernece_time = total_time/total_data
print(signal_infernece_time)

0.531015950642275


In [12]:
# new_df = pd.DataFrame(results, columns = ['predicted_tables','correct_tables','reference_tables','query','question','db_id'])
# new_df.to_csv('new_Prompt.csv', index=False)
#     print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
#     print("正确表名：",ref_tables)
#     print("预测表名：",total_selected_table)
#     print("############################################")
#     results.append([ref_tables,total_selected_table])
new_df = pd.DataFrame(results, columns = ['correct_tables','predicted_tables'])

In [13]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0

for index, row in new_df.iterrows():
    
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(", ")
    reference_tables = row['correct_tables'].split(", ")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1
    
    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracy
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

# new_df.to_csv("generated_schema_links.csv", index=False)

Total Accuracy: 0.7134831460674157
Filtered Accuracy: 0.7752808988764045
Average Precision: 0.8637640449438202
Average Recall: 0.8515917602996255


In [13]:
new_df.to_csv("generated_schema_links.csv", index=False)